In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import aiohttp
import asyncio
import os
import time
import shutil
import nest_asyncio; nest_asyncio.apply()
import pandas as pd
import numpy as np
from pathlib import Path
import geopandas as gpd

In [2]:
df = pd.read_csv('roadhome_for_datastore.csv')

Unique Stfid's and taking only first numbers of the stfid because first two represents state and next three reps parish.

In [3]:
stfids = set(df["STFID"])
stfidss = []
for stfid in stfids:
    stfidss.append(str(stfid)[:5])

In [4]:
unique_parish_stfid_codes = list(set(stfidss))

In [5]:
sites = []
for x in unique_parish_stfid_codes:
    url = f'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_{x}_tabblock00.zip'
    sites.append(url)

In [6]:
for site in sites:
    print(site)

https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22047_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22117_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22105_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22103_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22087_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22125_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22063_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22075_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22057_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22109_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2000/tl_2010_22071_tabblock00.zip
https://www2.census.gov/geo/tiger/TIGER2010

In [7]:
files =[]
files_l =[]
file_names = []
zip_folder = "location-detail-files"
extracted_folder = "extracted"
shp_folder = "shp_files"

if not os.path.exists('location-detail-files'):
    os.mkdir('location-detail-files')

if not os.path.exists('extracted'):
    os.mkdir('extracted')

if not os.path.exists('shp_files'):
    os.mkdir('shp_files')


In [8]:
def download_zipfile(url, destination):
    response = requests.get(url, stream=True)
    with open(destination, 'wb') as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)

In [9]:
def count_downloaded_files(folder):
    return len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

In [10]:
destination = "location-detail-files/"
for site in list(set(sites)):
    download_zipfile(site, destination + site.split('/')[-1])


In [11]:
len(sites), count_downloaded_files(zip_folder)


(37, 38)

In the above cell one count higher is because macOS creates default '.DS_Store' for all folders.

In [12]:
def unzip_and_extract_shp(zip_folder, extraction_folder):
    zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]
    
    for zip_file in zip_files:
        with zipfile.ZipFile(os.path.join(zip_folder, zip_file), 'r') as zip_ref:
            zip_ref.extractall(extraction_folder)

    shp_files = [f for f in os.listdir(extraction_folder) if f.endswith('.shp')]
    
    return shp_files

In [13]:
zip_folder = "location-detail-files/"
extraction_folder = "extracted-files/"

shp_files = unzip_and_extract_shp(zip_folder, extraction_folder)

print("Extracted .shp files:", shp_files, )

Extracted .shp files: ['tl_2010_22063_tabblock00.shp', 'tl_2010_22089_tabblock00.shp', 'tl_2010_22007_tabblock00.shp', 'tl_2010_22071_tabblock00.shp', 'tl_2010_22103_tabblock00.shp', 'tl_2010_22093_tabblock00.shp', 'tl_2010_22087_tabblock00.shp', 'tl_2010_22117_tabblock00.shp', 'tl_2010_22001_tabblock00.shp', 'tl_2010_22095_tabblock00.shp', 'tl_2010_22077_tabblock00.shp', 'tl_2010_22105_tabblock00.shp', 'tl_2010_22019_tabblock00.shp', 'tl_2010_22097_tabblock00.shp', 'tl_2010_22011_tabblock00.shp', 'tl_2010_22075_tabblock00.shp', 'tl_2010_22085_tabblock00.shp', 'tl_2010_22003_tabblock00.shp', 'tl_2010_22115_tabblock00.shp', 'tl_2010_22099_tabblock00.shp', 'tl_2010_22101_tabblock00.shp', 'tl_2010_22109_tabblock00.shp', 'tl_2010_22091_tabblock00.shp', 'tl_2010_22005_tabblock00.shp', 'tl_2010_22113_tabblock00.shp', 'tl_2010_22055_tabblock00.shp', 'tl_2010_22039_tabblock00.shp', 'tl_2010_22023_tabblock00.shp', 'tl_2010_22047_tabblock00.shp', 'tl_2010_22121_tabblock00.shp', 'tl_2010_22037_ta

In [14]:
len(shp_files)

37

In [15]:
df = pd.read_csv('roadhome_for_datastore.csv')

Check out this link for data dict for .shp files 
https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2009/TGRSHP09AA.pdf
<br>I have downloded only 
<br>Block County-based Shapefile (Census 2000)
<br>File Name: tl_2009_state-county FIPS_tabblock00.shp example : tl_2010_22125_tabblock00.shp

In [16]:

shp_folder = "extracted-files/"
gdf_list = []
for file_name in os.listdir(shp_folder):
    if file_name.endswith(".shp"):
        file_path = os.path.join(shp_folder, file_name)
        gdf = gpd.read_file(file_path)
        gdf_list.append(gdf)

# Concatenate GeoPandas DataFrames by rows
final_gdf = pd.concat(gdf_list, ignore_index=True)


Deriving a new attribute CENSUSTRACK(STATEFP00 + COUNTYFP00 + TRACTCE00) which is same as "Census Track" in road_home_dataset

This attribute is useful when STFID of road_home dataset and BLKIDFP00 of GDF are not same

In [17]:
final_gdf["CENSUSTRACK"] = final_gdf["STATEFP00"].astype(str) + final_gdf["COUNTYFP00"].astype(str) + final_gdf["TRACTCE00"].astype(str)

In [18]:
final_gdf.head()

,STATEFP00,COUNTYFP00,TRACTCE00,BLOCKCE00,BLKIDFP00,NAME00,MTFCC00,UR00,UACE00,FUNCSTAT00,ALAND00,AWATER00,INTPTLAT00,INTPTLON00,geometry,CENSUSTRACK
0,22,063,040302,2010,220630403022010,Block 2010,G5040,R,NaN,S,2735044,0,+30.5459148,-090.8537834,"POLYGON ((-90.85996 30.54781, -90.85991 30.547...",22063040302
1,22,063,040400,2028,220630404002028,Block 2028,G5040,U,05680,S,20072,0,+30.5008972,-090.8643358,"POLYGON ((-90.86422 30.50030, -90.86434 30.500...",22063040400
2,22,063,040400,2031,220630404002031,Block 2031,G5040,U,05680,S,46482,0,+30.5039122,-090.8660533,"POLYGON ((-90.86414 30.50399, -90.86416 30.503...",22063040400
3,22,063,040400,4018,220630404004018,Block 4018,G5040,U,05680,S,20248,0,+30.4938599,-090.8601758,"POLYGON ((-90.86091 30.49340, -90.86107 30.493...",22063040400
4,22,063,040400,4026,220630404004026,Block 4026,G5040,U,05680,S,183748,0,+30.4870071,-090.8568332,"POLYGON ((-90.85887 30.48707, -90.85885 30.487...",22063040400


In [19]:
df.head()

,Unnamed: 0,Structure Type,GIS City,GIS State,GIS Zip,PARISH,Closing Option,TOTAL_CLOSING_AMOUNT,Total CG Amount,Total ACG Amunt,...,Current Legal Fees,Closing Damage Assessment,PSV at Closing,"Closed file - PSV Option 1 With No ACG -excluding Sold Home, Inelig, Deceased and files closed with RHEI & IMM only -23,912","Closed File - Option 2/3, Sold Home, Ineligible, Deceased, and Closed with RHEI and IMM Only",Closed with Approved Unmet Needs (Y/N),Applicant With Current Insurance (Private and/or Flood) Y/N,Closing Total DOB Amount,Difference between Current PSV and Current Damage Assessment Type 2 (col AG - col AC,ARS File (Yes/No)
0,0,Single (including mobile home),NEW ORLEANS,LA,70128-3402,Orleans,1.0,70094.69,70094.69,0.00,...,2448.24,153738.26,216000.0,N,N,N,Y,53603.00,-62261.74,N
1,1,Single (including mobile home),NEW ORLEANS,LA,70127-1426,Orleans,1.0,50254.60,50254.60,0.00,...,0.00,163850.00,90000.0,Y,N,N,Y,18193.43,141371.15,N
2,2,Single (including mobile home),HARVEY,LA,70058-1934,Jefferson,1.0,60090.59,22590.59,0.00,...,0.00,44807.63,130000.0,N,N,Y,Y,22217.04,-85192.37,N
3,3,Single (including mobile home),NEW ORLEANS,LA,70127-1415,Orleans,1.0,46500.00,9000.00,0.00,...,0.00,171325.00,109252.0,Y,N,Y,Y,91000.00,141920.37,N
4,4,Single (including mobile home),NEW ORLEANS,LA,70125-2954,Orleans,1.0,150000.00,97079.05,22920.95,...,0.00,400750.00,190000.0,N,N,N,Y,92920.95,317507.01,N


First merging latitude and longitude based on stfid

In [27]:
final_gdf["BLKIDFP00"] = final_gdf["BLKIDFP00"].astype(int)
final_gdf["CENSUSTRACK"] = final_gdf["CENSUSTRACK"].astype(int)
# df1 = df.merge(gdf[["BLKIDFP00", "INTPTLAT00", "INTPTLON00"]], left_on="STFID", right_on="BLKIDFP00", how = "left")

first_merge_df = df.merge(final_gdf[["BLKIDFP00", "INTPTLAT00", "INTPTLON00"]], left_on="STFID", right_on="BLKIDFP00", how = "inner")

In [35]:
df[["STFID"]].nunique()

STFID    22544
dtype: int64

In [37]:
df.isna().sum()

Unnamed: 0                                                                                                                         0
Structure Type                                                                                                                     0
GIS City                                                                                                                           0
GIS State                                                                                                                          0
GIS Zip                                                                                                                            0
PARISH                                                                                                                             0
Closing Option                                                                                                                     0
TOTAL_CLOSING_AMOUNT                                                 

In [36]:
first_merge_df.isna().sum()

Unnamed: 0                                                                                                                         0
Structure Type                                                                                                                     0
GIS City                                                                                                                           0
GIS State                                                                                                                          0
GIS Zip                                                                                                                            0
PARISH                                                                                                                             0
Closing Option                                                                                                                     0
TOTAL_CLOSING_AMOUNT                                                 

In [38]:
first_merge_df.to_csv("road_home_with_lat_lon.csv")

In [40]:
final_gdf["BLKIDFP00"].nunique(), len(final_gdf)

(93321, 93321)

In [41]:
df["STFID"].nunique(), len(df)

(22544, 130053)

Exploration

In [45]:
df["Structure Type"].value_counts()

Single (including mobile home)           105552
Duplex (with one owner-occupied unit)      8581
Mobile Home on Leased Land                 8065
Mobile Home                                5808
Townhouse                                   864
Condominium                                 551
Single Family on Leased Land                524
Single                                      107
Ineligible                                    1
Name: Structure Type, dtype: int64